In [ ]:
import os
import pandas as pd
import numpy as np
import math

# Definir funciones necesarias
def vector_magnitude(v):
    return math.sqrt(v[0]**2 + v[1]**2)

def dot_product(v1, v2):
    return v1[0]*v2[0] + v1[1]*v2[1]

def angle_between_vectors(v1, v2):
    if vector_magnitude(v1) == 0 or vector_magnitude(v2) == 0:
        return None  # Ángulo indefinido para vectores cero
    cos_theta = dot_product(v1, v2) / (vector_magnitude(v1) * vector_magnitude(v2))
    cos_theta = min(1.0, max(-1.0, cos_theta))  # Limitar el valor para evitar errores de redondeo
    angle = math.acos(cos_theta)
    angle_degrees = math.degrees(angle)
    return angle_degrees

def calculate_features(df):
    numeric_columns = [
        'ojo izquierdo_x', 'ojo izquierdo_y', 'ojo derecho_x', 'ojo derecho_y', 
        'vejiga anterior_x', 'vejiga anterior_y', 'vejiga posterior_x', 'vejiga posterior_y', 
        'cola 1_x', 'cola 1_y', 'cola 2_x', 'cola 2_y', 
        'cola 3_x', 'cola 3_y', 'cola 4_x', 'cola 4_y', 
        'cola 5_x', 'cola 5_y', 'cola 6_x', 'cola 6_y'
    ]
    
    for column in numeric_columns:
        df[column] = pd.to_numeric(df[column], errors='coerce')
    
    df['eye_mean_x'] = (df['ojo izquierdo_x'] + df['ojo derecho_x']) / 2
    df['eye_mean_y'] = (df['ojo izquierdo_y'] + df['ojo derecho_y']) / 2
    df['tail_mean_x'] = (df['cola 5_x'] + df['cola 6_x']) / 2
    df['tail_mean_y'] = (df['cola 5_y'] + df['cola 6_y']) / 2
    df['head_velocity'] = np.sqrt((df['eye_mean_x'].diff())**2 + (df['eye_mean_y'].diff())**2)
    df['tail_velocity'] = np.sqrt((df['tail_mean_x'].diff())**2 + (df['tail_mean_y'].diff())**2)
    
    current_head_vector = np.array([df["vejiga posterior_x"] - df["eye_mean_x"], df["vejiga posterior_y"] - df["eye_mean_y"]]).T
    magnitudes = np.linalg.norm(current_head_vector, axis=1)
    dot_products = np.sum(current_head_vector[:-1] * current_head_vector[1:], axis=1)
    magnitude_pairs = magnitudes[:-1] * magnitudes[1:]
    angles = np.arccos(dot_products / magnitude_pairs)
    angles = np.concatenate([angles, [angles[-1]]])
    df['angular_velocity'] = angles
    
    tail_angles = []
    for i in range(1, 5):
        vec1 = df[f'cola {i}_x'] - df[f'cola {i+1}_x'], df[f'cola {i}_y'] - df[f'cola {i+1}_y']
        vec2 = df[f'cola {i+1}_x'] - df[f'cola {i+2}_x'], df[f'cola {i+1}_y'] - df[f'cola {i+2}_y']
        angle = np.arctan2(vec2[1], vec2[0]) - np.arctan2(vec1[1], vec1[0])
        tail_angles.append(angle)
    df['sum_tail_angles'] = np.sum(tail_angles, axis=0)
    
    df['tail6_velocity'] = np.sqrt((df['cola 3_x'].diff())**2 + (df['cola 3_y'].diff())**2)
    df['head_tail_distance'] = np.sqrt((df['eye_mean_x'] - df['tail_mean_x'])**2 + (df['eye_mean_y'] - df['tail_mean_y'])**2)
    df['head_acceleration'] = df['head_velocity'].diff().fillna(0)
    df['tail_acceleration'] = df['tail_velocity'].diff().fillna(0)
    df['angular_acceleration'] = df['angular_velocity'].diff().fillna(0)
    df['head_jerk'] = df['head_acceleration'].diff().fillna(0)
    df['tail_jerk'] = df['tail_acceleration'].diff().fillna(0)
    
    curvatures = []
    for i in range(1, 5):
        x1, y1 = df[f'cola {i}_x'], df[f'cola {i}_y']
        x2, y2 = df[f'cola {i+1}_x'], df[f'cola {i+1}_y']
        x3, y3 = df[f'cola {i+2}_x'], df[f'cola {i+2}_y']
        numerator = (x1 - x2) * (y2 - y3) - (y1 - y2) * (x2 - x3)
        denominator = np.sqrt(((x1 - x2)**2 + (y1 - y2)**2) * ((x2 - x3)**2 + (y2 - y3)**2) * ((x1 - x3)**2 + (y1 - y3)**2))
        curvature = numerator / denominator
        curvatures.append(curvature)
    df['sum_curvature'] = np.sum(curvatures, axis=0)
    
    delta_x = df['eye_mean_x'].diff()
    delta_y = df['eye_mean_y'].diff()
    heading_x = df['vejiga posterior_x'] - df['eye_mean_x']
    heading_y = df['vejiga posterior_y'] - df['eye_mean_y']
    df['angle_velocity_heading'] = np.arccos((delta_x * heading_x + delta_y * heading_y) / 
                                              (np.sqrt(delta_x**2 + delta_y**2) * np.sqrt(heading_x**2 + heading_y**2)))
    df['angle_velocity_heading'] = df['angle_velocity_heading'].fillna(0)
    df['curvature_rate'] = df['sum_curvature'].diff().fillna(0)

    return df

# Definir la carpeta que contiene los archivos Excel
input_folder = r'C:\Users\Usuario\Desktop\Prueba7'

# Iterar sobre todos los archivos en la carpeta
for filename in os.listdir(input_folder):
    if filename.endswith('.xlsx'):
        archivo = os.path.join(input_folder, filename)

        df = pd.read_excel(archivo)

        # Calcular las características
        features_df = calculate_features(df)

        # Crear un DataFrame con los promedios
        promedios_df = pd.DataFrame({
            'Variable': [
                'eye_mean_x', 'eye_mean_y', 'tail_mean_x', 'tail_mean_y', 'head_velocity', 'tail_velocity', 
                'angular_velocity', 'sum_tail_angles', 'tail6_velocity', 'head_tail_distance', 'head_acceleration', 
                'tail_acceleration', 'angular_acceleration', 'head_jerk', 'tail_jerk', 'sum_curvature', 
                'angle_velocity_heading', 'curvature_rate'
            ],
            'Promedio': [
                features_df['eye_mean_x'].mean(), features_df['eye_mean_y'].mean(), features_df['tail_mean_x'].mean(), 
                features_df['tail_mean_y'].mean(), features_df['head_velocity'].mean(), features_df['tail_velocity'].mean(), 
                features_df['angular_velocity'].mean(), features_df['sum_tail_angles'].mean(), features_df['tail6_velocity'].mean(), 
                features_df['head_tail_distance'].mean(), features_df['head_acceleration'].mean(), features_df['tail_acceleration'].mean(), 
                features_df['angular_acceleration'].mean(), features_df['head_jerk'].mean(), features_df['tail_jerk'].mean(), 
                features_df['sum_curvature'].mean(), features_df['angle_velocity_heading'].mean(), features_df['curvature_rate'].mean()
            ]
        })

        # Guardar los resultados en una nueva hoja del archivo Excel
        with pd.ExcelWriter(archivo, mode='a', engine='openpyxl') as writer:
            promedios_df.to_excel(writer, sheet_name='Promedios', index=False)

print("Promedios calculados y guardados en una nueva hoja llamada 'Promedios'.")
